# Multiobjective Portfolio Optimization

**Objective:** This notebook demonstrates how to use multiobjective optimization techniques to find optimal asset allocations for a portfolio. We will be using historical stock price data and the `pymoo` library to find a set of Pareto-optimal portfolios.

### 0. Multiobjective Optimization

Read [Mario meets Pareto](https://www.mayerowitz.io/blog/mario-meets-pareto) to understand multiobjective optimization.

### 1. Setup and Data Loading

First, install `pymoo`

In [ ]:
! pip install pymoo

Import the necessary libraries:

In [17]:
import pandas as pd
import numpy as np
from pymoo.util.remote import Remote
import matplotlib.pyplot as plt

Next, configure `Plotly` as the default plotting backend for `Pandas`:

In [13]:
pd.options.plotting.backend = "plotly"

Download the example dataset provided by pymoo:

In [ ]:
file = Remote.get_instance().load("examples", "portfolio_allocation.csv", to=None)
df = pd.read_csv(file, parse_dates=True, index_col="date")

The dataset portfolio_allocation.csv contains historical adjusted closing prices for multiple tickers, indexed by date. We will refer to df.columns (e.g., GOOG, AAPL) as the asset tickers in our portfolio.

### 2. Initial Data Visualization

Visualize the price series for all tickers:

In [ ]:
df.plot()

*Note:* The index is business days (trading days only). There are no weekend rows.

In [ ]:
df.head()

This plot shows the historical price evolution of each stock. Note that raw prices exhibit non-stationary behavior, so we typically work with returns rather than prices.

Calculate daily returns (percentage change) and drop any rows where all values are missing:

In [ ]:
returns = df.pct_change().dropna(how="all")
returns.plot()

Returns appear more "stationary" (they resemble random noise), which is why they are used for risk and return calculations.

### 3. Estimating Expected Investment Returns

We compute the annualized expected return $\mu_i$ for each asset $i$ using geometric (compound) returns:

In [15]:
mu = (1 + returns).prod() ** (252 / returns.count()) - 1

**Explanation of the formula:**
1. $(1 + r_t)$ for each return $r_t$ gives the daily growth factor.
2. $\prod_{t=1}^{T}(1 + r_t)$ computes the total growth factor over $T$ trading days.
3. Dividing the exponent by $T$ and multiplying by 252 (approximate trading days per year, we need to take weekends into account) annualizes this compound growth:
$$\mu_i = \left(\prod_{t=1}^{T}(1 + r_{i,t})\right)^{\frac{252}{T}} - 1$$
4. Subtracting 1 converts the compound multiplier back to a percentage return.

Plot the annualized returns for each asset:

In [ ]:
labels = df.columns

fig, ax = plt.subplots(figsize=(10, 5))
k = np.arange(len(mu))
ax.bar(k, mu)
ax.set_xticks(k, labels, rotation = 90)
plt.show()

### 4. Estimating Risk and Correlation

Calculate the annualized covariance matrix $\Sigma$ of returns:

In [25]:
cov = returns.cov() * 252 # Covariance scaled to annual

Compute and visualize the correlation between asset returns:

In [ ]:
f = plt.figure(figsize=(10, 10))
plt.matshow(returns.corr(), fignum=f.number)
plt.xticks(k, labels, fontsize=12, rotation=90)
plt.yticks(k, labels, fontsize=12)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

The covariance tells us how two assets co-vary (in units of variance). The correlation is simply the covariance standardized by the product of standard deviations. Here, we plot the correlation so that values lie between −1 and +1, making it easier to spot diversification benefits.

A lower correlation between two assets suggests they provide better diversification benefits. You can inspect this matrix to identify pairs of assets operating in different industries (e.g., technology vs. consumer staples) with low correlations.

In [ ]:
mu, cov = mu.to_numpy(), cov.to_numpy()

### 5. Defining the Optimization Problem

We construct a multiobjective optimization problem with two objectives:
1. Minimize portfolio risk.
2. Maximize portfolio return (by minimizing negative return).

### Mathematical Formulation of the Portfolio Problem
Consider a portfolio of $n$ assets. We denote:

- $x = (x_1, x_2, \ldots, x_n)^\top$ as the vector of portfolio weights, where $x_i$ is the fraction of total capital invested in asset $i$.
- $\mu = (\mu_1, \mu_2, \ldots, \mu_n)^\top$ as the vector of annualized expected returns for each asset.
- $\Sigma \in \mathbb{R}^{n \times n}$ as the annualized covariance matrix of asset returns.
- $r_f$ as the annual risk-free rate (default $r_f = 0.02$).

### Objective Functions

We formulate a multi-objective problem with two objectives:

1. **Minimize portfolio risk (annualized volatility)**  
   Let
   $$f_1(x) \;=\; \underbrace{\sqrt{x^\top\,\Sigma\,x}}_{\text{portfolio standard deviation}}.$$
   This quantity is the annualized standard deviation of the portfolio's returns.

2. **Maximize portfolio return**  
   Equivalently, minimize the negative of the expected return:
   $$f_2(x) \;=\; -\,\underbrace{\bigl(\mu^\top\,x\bigr)}_{\text{expected annual return}}.$$
   Since most multi-objective solvers assume a minimization format, we define the second objective as $-\,\mu^\top x$.

Hence, the multi-objective vector is

$$F(x) \;=\; \bigl[f_1(x),\,f_2(x)\bigr]
\;=\;\Bigl[\sqrt{x^\top\Sigma\,x},\;-\mu^\top x\Bigr].$$

### Constraints

1. **Budget Constraint (weights sum to 1):**  
   $$\sum_{i=1}^{n} x_i = 1.$$

2. **Box Constraints (no short-selling, no leverage):**  
   $$0 \;\leq\; x_i \;\leq\; 1, 
   \quad 
   \forall\,i = 1,2,\ldots,n.$$

3. **Practical Thresholding (small-weight trimming):**  
   In practice, any $x_i < 0.001$ (i.e., below 0.1% of the portfolio) is set to zero and then the remaining weights are renormalized to sum to 1. Mathematically, this means:
   $$\tilde{x}_i =
   \begin{cases}
   0, & x_i < 10^{-3},\\
   x_i, & x_i \geq 10^{-3},
   \end{cases}
   \quad
   \text{then renormalize } \tilde{x} \text{ so that } \sum_{i} \tilde{x}_i = 1.$$
   This ensures that very small positions are eliminated and the budget constraint is restored.

### Full Optimization Problem

Putting everything together, we can write:

$$\begin{aligned}
\text{Find } x \in \mathbb{R}^n \quad &\text{that minimizes } 
\;F(x) = \Bigl[\sqrt{x^\top \Sigma\,x},\, -\,\mu^\top x\Bigr],\\
\text{subject to:}
\quad
&\sum_{i=1}^n x_i = 1,\\
&0 \;\leq\; x_i \;\leq\; 1, \quad i = 1,\ldots,n.
\end{aligned}$$

After each candidate $x$ is generated by the optimizer, a repair step enforces:

1. $x_i \leftarrow 0$ for all $i$ such that $x_i < 10^{-3}$.
2. $x \leftarrow \displaystyle \frac{x}{\sum_{i=1}^n x_i}$ to restore $\sum_{i} x_i = 1$.

In [3]:
from pymoo.core.problem import ElementwiseProblem
import numpy as np
from typing import Any

class PortfolioProblem(ElementwiseProblem):
    """
    Portfolio optimization problem for multi-objective optimization.
    
    This class defines a portfolio optimization problem that minimizes risk (volatility)
    while maximizing expected return.
    """

    def __init__(self, mu: np.ndarray, cov: np.ndarray, **kwargs: Any) -> None:
        """
        Initialize the portfolio optimization problem.
        
        Args:
            mu: Expected returns for each asset (1D array)
            cov: Covariance matrix of asset returns (2D array)
            **kwargs: Additional arguments passed to parent class
        """
        super().__init__(n_var=len(df.columns), n_obj=2, xl=0.0, xu=1.0, **kwargs)
        self.mu = mu
        self.cov = cov

    def _evaluate(self, x: np.ndarray, out: dict[str, Any], *args: Any, **kwargs: Any) -> None:
        """
        Evaluate portfolio performance for given weights.
        
        Args:
            x: Portfolio weights (must sum to 1)
            out: Output dictionary to store objective values and metrics
            *args: Additional positional arguments
            **kwargs: Additional keyword arguments
        """
        # TODO: Implement F function
        # Note: We'll handle constraints in the repair method. Do not worry about constraints here.
        raise NotImplementedError("Implement this method!")
        # Note: This method do not return anything.
        # Instead it should store the objectives in out["F"]
        out["F"] = [f1, f2]

### Handling Constraints

We handle constraints by applying repair function that repairs solutions that are not feasible.

In [4]:
from pymoo.core.repair import Repair


class PortfolioRepair(Repair):
    """
    Portfolio repair mechanism for multi-objective optimization.

    This repair operator ensures that portfolio weights satisfy practical constraints:
    1. Portfolio weights sum to 100% (normalized to 1.0)
    2. Very small weights (< 0.1%) are set to zero to avoid impractical allocations

    The rationale is that investment weights must total 100% and very small fractions
    of investment don't make practical sense in real portfolio management.
    """

    def _do(
        self, problem: ElementwiseProblem, X: np.ndarray, **kwargs: Any
    ) -> np.ndarray:
        """
        Repair portfolio weights to satisfy normalization and minimum threshold constraints.

        Args:
            problem: The optimization problem instance (unused but required by interface)
            X: Portfolio weight matrix where each row represents a portfolio solution
            **kwargs: Additional keyword arguments

        Returns:
            Repaired portfolio weights matrix with proper normalization and thresholding
        """
        # X is a 2D array: each row is a candidate weight vector
        # For each row:
        #   a) Set weights < 0.001 to zero
        #   b) Make sure that sum(weights) == 1.0
        #   c) Return the repaired X matrix
        raise NotImplementedError("Implement this method!")

### 6. Running the Multiobjective Optimization

We use the SMS-EMOA algorithm from pymoo to obtain the Pareto front of optimal trade-offs between risk and return.

In [5]:
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.optimize import minimize

problem = PortfolioProblem(mu, cov)

algorithm = SMSEMOA(repair=PortfolioRepair())

res = minimize(problem, algorithm, seed=1, verbose=False)

Upon convergence, `res.opt.get("X", "F")` returns:
- X: Pareto-optimal weight matrices (each row is a portfolio).
- F: Corresponding objective values ([risk, –return]).

In [ ]:
X, F = res.opt.get("X", "F")
F = F * [1, -1]

plt.scatter(F[:, 0], F[:, 1], facecolor="none", edgecolors="blue", alpha=0.5, label="Pareto-Optimal Portfolio")
plt.scatter(cov.diagonal() ** 0.5, mu, facecolor="none", edgecolors="black", s=30, label="Asset")
plt.legend()
plt.xlabel("expected volatility")
plt.ylabel("expected return")
plt.show()

In this figure, each blue point is a portfolio on the Pareto front. The black points are the individual assets (their own risk–return coordinates). Notice that some portfolios lie ‘above’ all single assets (they achieve higher return for the same risk).

### Exercise 1

Visualize `X` using imshow. Analyse and identify which tickers have non‐zero weight.

### Exercise 2 
We want  to choose a single portfolio from the Pareto front. For this purpose we can use another metric -- Sharpe ratio:
- Compute Sharpe ratio for each Pareto portfolio:  
  $$\text{Sharpe}_j = \frac{\mu^\top x^{(j)} - r_f}{\sqrt{x^{(j)T} \Sigma x^{(j)}}}$$  
  Where $r_f = 0.02$.  
- Plot Sharpe vs. risk or Sharpe vs. return, and identify the portfolio that maximizes Sharpe.  